## ClimaLSM Compatible DALEC Model

**By Jianing Fang (jf3423@columbia.edu), last revision August 2022**

In [2]:
using OrdinaryDiffEq: solve, DiscreteProblem, FunctionMap
using DataFrames
using DALEC
using Plots
Plots.pyplot()

SyntaxError: invalid syntax (596941646.py, line 1)